In [ ]:
import datashader as ds
import datashader.transfer_functions as tf
import datashader.glyphs
from datashader import reductions
from datashader.core import bypixel
from datashader.utils import lnglat_to_meters as webm, export_image
from datashader.colors import colormap_select, Greys9, viridis, inferno
import copy

import warnings
warnings.filterwarnings('ignore')

from pyproj import Proj, transform
import numpy as np
import pandas as pd
import urllib
import json
import datetime
import colorlover as cl

import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools

# from shapely.geometry import Point, Polygon, shape
# In order to get shapley, you'll need to run [pip install shapely.geometry] from your terminal

from functools import partial

from IPython.display import GeoJSON

py.init_notebook_mode()

For module 2 we'll be looking at techniques for dealing with big data. In particular binning strategies and the datashader library (which possibly proves we'll never need to bin large data for visualization ever again.)

To demonstrate these concepts we'll be looking at the PLUTO dataset put out by New York City's department of city planning. PLUTO contains data about every tax lot in New York City.

PLUTO data can be downloaded from [here](https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page). Unzip them to the same directory as this notebook, and you should be able to read them in using this (or very similar) code. Also take note of the data dictionary, it'll come in handy for this assignment.

In [ ]:
# Code to read in v17, column names have been updated (without upper case letters) for v18

# bk = pd.read_csv('PLUTO17v1.1/BK2017V11.csv')
# bx = pd.read_csv('PLUTO17v1.1/BX2017V11.csv')
# mn = pd.read_csv('PLUTO17v1.1/MN2017V11.csv')
# qn = pd.read_csv('PLUTO17v1.1/QN2017V11.csv')
# si = pd.read_csv('PLUTO17v1.1/SI2017V11.csv')

# ny = pd.concat([bk, bx, mn, qn, si], ignore_index=True)

ny = pd.read_csv('nyc_pluto_20v5_csv/pluto.csv')


# Getting rid of some outliers
ny = ny[(ny['yearbuilt'] > 1850) & (ny['yearbuilt'] < 2020) & (ny['numfloors'] != 0)]

I'll also do some prep for the geographic component of this data, which we'll be relying on for datashader.

You're not required to know how I'm retrieving the lattitude and longitude here, but for those interested: this dataset uses a flat x-y projection (assuming for a small enough area that the world is flat for easier calculations), and this needs to be projected back to traditional lattitude and longitude.

In [ ]:
# wgs84 = Proj("+proj=longlat +ellps=GRS80 +datum=NAD83 +no_defs")
# nyli = Proj("+proj=lcc +lat_1=40.66666666666666 +lat_2=41.03333333333333 +lat_0=40.16666666666666 +lon_0=-74 +x_0=300000 +y_0=0 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs")
# ny['xcoord'] = 0.3048*ny['xcoord']
# ny['ycoord'] = 0.3048*ny['ycoord']
# ny['lon'], ny['lat'] = transform(nyli, wgs84, ny['xcoord'].values, ny['ycoord'].values)

# ny = ny[(ny['lon'] < -60) & (ny['lon'] > -100) & (ny['lat'] < 60) & (ny['lat'] > 20)]

#Defining some helper functions for DataShader
background = "black"
export = partial(export_image, background = background, export_path="export")
cm = partial(colormap_select, reverse=(background!="black"))

## Part 1: Binning and Aggregation

Binning is a common strategy for visualizing large datasets. Binning is inherent to a few types of visualizations, such as histograms and [2D histograms](https://plot.ly/python/2D-Histogram/) (also check out their close relatives: [2D density plots](https://plot.ly/python/2d-density-plots/) and the more general form: [heatmaps](https://plot.ly/python/heatmaps/).

While these visualization types explicitly include binning, any type of visualization used with aggregated data can be looked at in the same way. For example, lets say we wanted to look at building construction over time. This would be best viewed as a line graph, but we can still think of our results as being binned by year:

In [ ]:
trace = go.Scatter(
    # I'm choosing BBL here because I know it's a unique key.
    x = ny.groupby('yearbuilt').count()['bbl'].index,
    y = ny.groupby('yearbuilt').count()['bbl']
)

layout = go.Layout(
    xaxis = dict(title = 'Year Built'),
    yaxis = dict(title = 'Number of Lots Built')
)

fig = go.FigureWidget(data = [trace], layout = layout)

fig

Something looks off... You're going to have to deal with this imperfect data to answer this first question. 

But first: some notes on pandas. Pandas dataframes are a different beast than R dataframes, here are some tips to help you get up to speed:

---

Hello all, here are some pandas tips to help you guys through this homework:

[Indexing and Selecting](https://pandas.pydata.org/pandas-docs/stable/indexing.html): .loc and .iloc are the analogs for base R subsetting, or filter() in dplyr

[Group By](https://pandas.pydata.org/pandas-docs/stable/groupby.html):  This is the pandas analog to group_by() and the appended function the analog to summarize(). Try out a few examples of this, and display the results in Jupyter. Take note of what's happening to the indexes, you'll notice that they'll become hierarchical. I personally find this more of a burden than a help, and this sort of hierarchical indexing leads to a fundamentally different experience compared to R dataframes. Once you perform an aggregation, try running the resulting hierarchical datafrome through a [reset_index()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html).

[Reset_index](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html): I personally find the hierarchical indexes more of a burden than a help, and this sort of hierarchical indexing leads to a fundamentally different experience compared to R dataframes. reset_index() is a way of restoring a dataframe to a flatter index style. Grouping is where you'll notice it the most, but it's also useful when you filter data, and in a few other split-apply-combine workflows. With pandas indexes are more meaningful, so use this if you start getting unexpected results.

Indexes are more important in Pandas than in R. If you delve deeper into the using python for data science, you'll begin to see the benefits in many places (despite the personal gripes I highlighted above.) One place these indexes come in handy is with time series data. The pandas docs have a [huge section](http://pandas.pydata.org/pandas-docs/stable/timeseries.html) on datetime indexing. In particular, check out [resample](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html), which provides time series specific aggregation.

[Merging, joining, and concatenation](https://pandas.pydata.org/pandas-docs/stable/merging.html): There's some overlap between these different types of merges, so use this as your guide. Concat is a single function that replaces cbind and rbind in R, and the results are driven by the indexes. Read through these examples to get a feel on how these are performed, but you will have to manage your indexes when you're using these functions. Merges are fairly similar to merges in R, similarly mapping to SQL joins.

Apply: This is explained in the "group by" section linked above. These are your analogs to the plyr library in R. Take note of the lambda syntax used here, these are anonymous functions in python. Rather than predefining a custom function, you can just define it inline using lambda.

Browse through the other sections for some other specifics, in particular reshaping and categorical data (pandas' answer to factors.) Pandas can take a while to get used to, but it is a pretty strong framework that makes more advanced functions easier once you get used to it. Rolling functions for example follow logically from the apply workflow (and led to the best google results ever when I first tried to find this out and googled "pandas rolling")

Google Wes Mckinney's book "Python for Data Analysis," which is a cookbook style intro to pandas. It's an O'Reilly book that should be pretty available out there.

---

### Question

After a few building collapses, the City of New York is going to begin investigating older buildings for safety. The city is particularly worried about buildings that were unusually tall when they were built, since best-practices for safety hadn’t yet been determined. Create a graph that shows how many buildings of a certain number of floors were built in each year (note: you may want to use a log scale for the number of buildings). Find a strategy to bin buildings (It should be clear 20-29-story buildings, 30-39-story buildings, and 40-49-story buildings were first built in large numbers, but does it make sense to continue in this way as you get taller?)

--------------------------
From the documentation, we know a few things about the variable **numfloors**:
* The number of full and partial stories starting from the ground floor, for the tallest building on the tax lot.
* Above ground basements are not included in the NUMBER OF FLOORS.
* A roof used for parking, farming, playground, etc. is not included in NUMBER OF FLOORS.
* If the NUMBER OF FLOORS is zero and the NUMBER OF BUILDINGS is greater than zero, then NUMBER OF FLOORS is not available for the tax lot.
* If the NUMBER OF FLOORS is zero and the NUMBER OF BUILDINGS is zero, then NUMBER OF FLOORS is not applicable for the tax lot.

First, we'll take a look at the summary statistics for the variable **numfloors** to see what the data looks like. We notice that most of the buildings have between 0 - 2.5 floors. When we plot this data in a histogram, we begin to see quite a few outliers - buildings that have an abnormally large number of floors. 

In [ ]:
# summary statistics
nySub = ny[['yearbuilt', 'numfloors', 'numbldgs']]
nySub['numfloors'].describe()

In [ ]:
# preliminary look at floors
import plotly.express as px
df = px.data.tips()
fig = px.histogram(nySub, x="numfloors", 
                   labels={
                     "numfloors": "Number of Floors",
                     "count": "Count"
                 },
                   title="Distribution of the number of floors in buildings")
fig.show()

Let's remove our outliers and look a little closer at the most common values in the dataset. We can see that with outliers excluded, buildings have between 1 and 4 floors. 

In [ ]:
# defining bounds for outliers
q1, q3= np.percentile(sorted(nySub['numfloors']),[25,75])
iqr = q3 - q1
lower_bound = q1 -(1.5 * iqr) 
upper_bound = q3 +(1.5 * iqr) 

# creating outlier column
conditions = [
    (nySub['numfloors'] < lower_bound),
    (nySub['numfloors'] > upper_bound)]

choices = [1,1]
nySub['outlier'] = np.select(conditions, choices, default=0)

In [ ]:
df = px.data.tips()
fig = px.histogram(nySub[nySub['outlier']==0], x="numfloors", 
                   labels={
                     "numfloors": "Number of Floors",
                     "count": "Count"
                 },
                   title="Distribution of the number of floors in buildings (No Outliers)")
fig.show()

Now we can look at the outliers. We only have a small number of data points past 100 floors, so once we reach this number, we'll bucket all floors together. In other words, our last bucket will be 100+ floors. 

In [ ]:
df = px.data.tips()
fig = px.histogram(nySub[nySub['outlier']==1], x="numfloors", 
                   labels={
                     "numfloors": "Number of Floors",
                     "count": "Count"
                 },
                   title="Distribution of the number of floors in buildings (Outliers only)")
fig.show()

In [ ]:
# define bins
bins = [0,10,20,30,40,50,60,70,80,90,100,max(nySub['numfloors'])]
labs = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101+']

nySub['bins'] = pd.cut(nySub['numfloors'],
                       bins = bins,
                       labels = labs,
                       include_lowest = True)

# group data and count for bins, log transform output
g1 = nySub.groupby(['yearbuilt', 'bins']).count()['numbldgs'].reset_index()
g1['numbldgs'] = np.log10(g1['numbldgs'])

In [ ]:
fig = px.bar(g1, x="yearbuilt", y="numbldgs", color="bins", 
             labels={
                     "yearbuilt": "Year Built",
                     "numbldgs": "Number of Buildings",
                     "bins": "Number of Stories"
                 },
             title="Distribution of Building Sizes by Year")
fig.show()

## Part 2: Datashader

Datashader is a library from Anaconda that does away with the need for binning data. It takes in all of your datapoints, and based on the canvas and range returns a pixel-by-pixel calculations to come up with the best representation of the data. In short, this completely eliminates the need for binning your data.

As an example, lets continue with our question above and look at a 2D histogram of YearBuilt vs NumFloors:

In [ ]:
yearbins = 200
floorbins = 200

yearBuiltCut = pd.cut(ny['yearbuilt'], np.linspace(ny['yearbuilt'].min(), ny['yearbuilt'].max(), yearbins))
numFloorsCut = pd.cut(ny['numfloors'], np.logspace(1, np.log(ny['numfloors'].max()), floorbins))

xlabels = np.floor(np.linspace(ny['yearbuilt'].min(), ny['yearbuilt'].max(), yearbins))
ylabels = np.floor(np.logspace(1, np.log(ny['numfloors'].max()), floorbins))

fig = go.FigureWidget(
    data = [
        go.Heatmap(z = ny.groupby([numFloorsCut, yearBuiltCut])['bbl'].count().unstack().fillna(0).values,
              colorscale = 'Greens', x = xlabels, y = ylabels)
    ]
)

fig

This shows us the distribution, but it's subject to some biases discussed in the Anaconda notebook [Plotting Perils](https://anaconda.org/jbednar/plotting_pitfalls/notebook). 

Here is what the same plot would look like in datashader:



In [ ]:
cvs = ds.Canvas(800, 500, x_range = (ny['yearbuilt'].min(), ny['yearbuilt'].max()), 
                                y_range = (ny['numfloors'].min(), ny['numfloors'].max()))
agg = cvs.points(ny, 'yearbuilt', 'numfloors')
view = tf.shade(agg, cmap = cm(Greys9), how='log')
export(tf.spread(view, px=2), 'yearvsnumfloors')

That's technically just a scatterplot, but the points are smartly placed and colored to mimic what one gets in a heatmap. Based on the pixel size, it will either display individual points, or will color the points of denser regions.

Datashader really shines when looking at geographic information. Here are the latitudes and longitudes of our dataset plotted out, giving us a map of the city colored by density of structures:

In [ ]:
NewYorkCity   = (( 913164.0,  1067279.0), (120966.0, 272275.0))
cvs = ds.Canvas(700, 700, *NewYorkCity)
agg = cvs.points(ny, 'xcoord', 'ycoord')
view = tf.shade(agg, cmap = cm(inferno), how='log')
export(tf.spread(view, px=2), 'firery')

Interestingly, since we're looking at structures, the large buildings of Manhattan show up as less dense on the map. The densest areas measured by number of lots would be single or multi family townhomes.

Unfortunately, Datashader doesn't have the best documentation. Browse through the examples from their [github repo](https://github.com/bokeh/datashader/tree/master/examples). I would focus on the [visualization pipeline](https://anaconda.org/jbednar/pipeline/notebook) and the [US Census](https://anaconda.org/jbednar/census/notebook) Example for the question below. Feel free to use my samples as templates as well when you work on this problem.

### Question

You work for a real estate developer and are researching underbuilt areas of the city. After looking in the [Pluto data dictionary](https://www1.nyc.gov/assets/planning/download/pdf/data-maps/open-data/pluto_datadictionary.pdf?v=17v1_1), you've discovered that all tax assessments consist of two parts: The assessment of the land and assessment of the structure. You reason that there should be a correlation between these two values: more valuable land will have more valuable structures on them (more valuable in this case refers not just to a mansion vs a bungalow, but an apartment tower vs a single family home). Deviations from the norm could represent underbuilt or overbuilt areas of the city. You also recently read a really cool blog post about [bivariate choropleth maps](http://www.joshuastevens.net/cartography/make-a-bivariate-choropleth-map/), and think the technique could be used for this problem.

Datashader is really cool, but it's not that great at labeling your visualization. Don't worry about providing a legend, but provide a quick explanation as to which areas of the city are overbuilt, which areas are underbuilt, and which areas are built in a way that's properly correlated with their land value.

--------------

Our bivariate choropleth map will visualize assessment of land vs assessment of structure. Since we are given the Land Assessment and the Total Assessment, we can back into the Structure Assessment.

In [ ]:
nya = ny[['assessland', 'assesstot', 'xcoord', 'ycoord']]
nya['assesstruc'] = nya['assesstot'] - nya['assessland']

We need to categorize each variable (land assessment and structure assessment) into 3 groups: low, medium, and high. The strategy will be to group the data into bins based on the 33rd, 66th, and 100th percentiles for each variable. 

In [ ]:
binstruc = [0, np.percentile(nya['assesstruc'], 100/3), np.percentile(nya['assesstruc'], (100/3) *2), max(nya['assesstruc'])]
binland = [0, np.percentile(nya['assessland'], 100/3), np.percentile(nya['assessland'], (100/3) *2), max(nya['assessland'])]

nya['valstruc'] = pd.cut(nya['assesstruc'],
                   bins = binstruc,
                   labels = ['A','B','C'],
                        include_lowest  = True)

nya['valland'] = pd.cut(nya['assessland'],
                   bins = binland,
                   labels = ['1','2','3'],
                        include_lowest = True)

Now that we have defined the individual categories, we can define the final, 9-color map for the data by combining the values.

In [ ]:
# https://stackoverflow.com/questions/39092067/pandas-dataframe-convert-column-type-to-string-or-categorical
nya['bi_class'] = nya['valstruc'].astype(str) + nya['valland'].astype(str)
nya['bi_class'] = pd.Categorical(nya['bi_class'])
nya.head()

We will use one of the color scheme suggested in the Bivariate Choropleth Map article:
* Variable 1: structure assessment (from left to right)
* Variable 2: land assessment (from bottom to top)

![title](pic.png)

The colors of interest to us are:
* **Blue**: Low structure assessment, but high land assessment (underbuilt)
* **Red**: High structure assessment, but low land assessment (overbuilt)
* **White**: Low structure and low land assessment (correlated with land value)
* **Brown**: High structure and high land assessment (correlated with land value)

In [ ]:
colormp = {"A1":"#FFF7EB", "B1": "#D7FCD8", "C1": "#A8E9AA",
"A2": "#FBD6D6" , "B2": "#B9D6EA" , "C2": "#A6D5AA",
"A3": "#FFBEBD", "B3": "#EDC9F1" , "C3": "#9899C5"}

colormp = {"A1":"#e8e8e8", "B1": "#e4acac", "C1": "#c85a5a",
"A2": "#b0d5df" , "B2": "#ad9ea5" , "C2": "#985356",
"A3": "#64acbe", "B3": "#627f8c" , "C3": "#574249"}

NewYorkCity   = (( 913164.0,  1067279.0), (120966.0, 272275.0))
cvs = ds.Canvas(700, 700, *NewYorkCity)
agg = cvs.points(nya, 'xcoord', 'ycoord', ds.count_cat('bi_class')) 
view = tf.shade(agg, color_key = colormp, how='log')

export(tf.spread(view, px=2), 'final')

Not the prettiest looking colormap, but it does highlight some interesting things. (You're not kidding that the documentation for datashader isn't great -- this took forever!)

Let's take a look at each individually: 
* **Blue - Underbuilt**: We see some darker blue areas in upper-mid Queens, by the LaGuardia Airport. The buildings themselves aren't worth too much, but they're in a prime location. This is an area of opportunity for an investor. 
* **Red - Overbuilt**: We see areas of underbuilt properties in upper Queens and Brooklyn, down through to Prospect Park. The buildings in these areas tend to be worth more than the land that they're on.  
* **Brown**: We can see that Manhattan is predominantly brown -- this means that most of structures are high-valued and on land that is also high-valued. This makes sense, as it's perhaps the most visited borough of the NYC region. 
* **White**: Much of Staten Island (more suburbs) have low valued land and structures.
